In [4]:
import os as os
import glob as glob
import shutil as shutil
import requests as req
import zipfile as zipfile
from datetime import datetime

In [5]:
def data_directories(**kwargs):
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    Args:
        directories (list): Lista do caminho dos diretorios a serem criados.
        
    Return:
        Cria os diretorios para armazenamento dos dados.  
    """

    # Diretorios para armazenamento dos dados coletados
    kwargs['directories'] = ['..\\data\\01-collected'
                            ,'..\\data\\02-cleaned'
                            ,'..\\data\\03-organized'
                            ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Inicio da criacao dos diretorios para armazenamento dos dados.')
    
    # Para cada diretorio na lista de diretorios
    for directory in kwargs['directories']:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Fim da criacao dos diretorios para armazenamento dos dados.')

In [6]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo, extrai os arquivos compactados, organiza-os em
    um unico diretorio e remove os arquivos que nao sao XML. Ao final do download, e verificado 
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)
    
    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Log: Mensagem de URL encontrada e de inicio do download
        print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo, organiza-os 
        # no diretorio e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # Log: Mensagem de inicio da organizacao dos arquivos
            print(str(datetime.now()) + ': Organizando os arquivos descompactados de: ' + kwargs['download_path'])
            
            # Move os arquivos em subdiretorios para o diretorio principal
            subdirList = glob.glob(kwargs['download_directory'] + '**\\')
            for subdir in subdirList:
                fileList = os.listdir(subdir)
                for file in fileList:
                    shutil.move(subdir + file, kwargs['download_directory'])
                os.rmdir(subdir)
            
            # Remove arquivos JPG inclusos no zip
            fileList = glob.glob(kwargs['download_directory'] + '*.jpg')
            for filePath in fileList:
                os.remove(filePath)
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])

In [7]:
def collect_data(**kwargs):
    """
    Coleta os dados de cada URL de referencia contida na lista informada.
    
    Args:
        url_list (list): Lista de URLs para download do DOU.
    """
    
    ### Define a data de referencia inicial
    ### kwargs['dt_ref'] = kwargs['dt_ini_ref']
    
    # Cria os diretorios para armazenamento dos dados
    data_directories(**kwargs)
    
    # Define o caminho completo para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    ### Define a URL base de busca dos arquivos para download
    ### kwargs['download_url'] = 'http://www.portaltransparencia.gov.br/download-de-dados/cpgf/'
    
    # Define o nome do arquivo para download
    kwargs['file_name'] = '_DOU'
    kwargs['file_ext'] = '.zip'
       
    # Para cada URL de referencia dentro da lista
    for i, url_ref in enumerate(kwargs['url_list']):
        
        # Prepara o nome do arquivo para download
        kwargs['download_file'] = str(i) + kwargs['file_name'] + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = url_ref
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)
        
        ### Atualiza o mes de refencia
        ### kwargs['dt_ref'] += relativedelta(months=1)

    

In [8]:
def main():
    """
    Coleta os dados do Diario Oficial da Uniao (DOU)
    Origem: Portal da Brasileiro de Dados Abertos (dados.gov.br)
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Lista de URLs para download dos arquivos zip do DOU 
    kwargs['url_list'] = [#'http://www.imprensanacional.gov.br/documents/20181/5431307/S03022018.zip/dcb5e806-3df2-474f-95f2-124383b7b57e'
                         #,'http://www.in.gov.br/documents/20181/8775202/S03032018/18338f4c-ecfa-4dc8-9782-96243050da15'
                         #,'http://portal.imprensanacional.gov.br/documents/20181/12529056/S03042018.zip/04cbbaa5-8791-4571-a573-9bea60896c49'
                         #,'http://www.imprensanacional.gov.br/documents/20181/17112825/S03052018.zip/1efdef62-64c5-4c45-ac22-f1a8738efd09'
                         #,'http://www.imprensanacional.gov.br/documents/20181/28342230/S03062018.zip/6ff68268-621e-4965-944a-f764c5477fa6'
                         #,'http://www.imprensanacional.gov.br/documents/20181/35749259/S03072018.zip/b8523228-0212-4072-97bd-1e6e68785383'
                         #,'http://www.imprensanacional.gov.br/documents/20181/39986130/S03082018.zip/7bfc3011-125b-4917-9ae6-86419f92d3bf'
                         #,'http://www.imprensanacional.gov.br/documents/20181/43717513/S03092018.zip/caf94f92-a7e9-4821-b0c1-766d42e1dec5'
                         #,'http://www.imprensanacional.gov.br/documents/20181/49533667/S03102018.zip/b92ee990-36e5-4360-a923-0861641cdf78'
                         #,'http://www.imprensanacional.gov.br/documents/49035712/50301464/S03112018.zip/3c7e3009-e2ca-43f6-b7f7-3af6e6d58385'
                         'http://www.imprensanacional.gov.br/documents/49035712/50301468/S03122018.zip/8d942918-1a12-458a-b5dc-91cfaa392435']
    
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2019-07-24 09:01:28.665751: Inicio da criacao dos diretorios para armazenamento dos dados.
2019-07-24 09:01:28.666747: Diretorio ..\data\01-collected ja existe.
2019-07-24 09:01:28.666747: Diretorio ..\data\02-cleaned ja existe.
2019-07-24 09:01:28.666747: Diretorio ..\data\03-organized ja existe.
2019-07-24 09:01:28.667745: Diretorio ..\data\04-standardized ja existe.
2019-07-24 09:01:28.667745: Fim da criacao dos diretorios para armazenamento dos dados.
2019-07-24 09:01:28.667745: Inicio do download do arquivo ..\data\01-collected\0_DOU.zip
2019-07-24 09:01:31.435672: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\0_DOU.zip
2019-07-24 09:03:18.323013: Descompactando o arquivo ..\data\01-collected\0_DOU.zip
2019-07-24 09:13:49.344517: Organizando os arquivos descompactados de: ..\data\01-collected\0_DOU.zip
2019-07-24 09:19:59.523714: Fim do download do arquivo ..\data\01-collected\0_DOU.zip
